<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# AI-First Finance

**Convolutional Neural Networks**

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## Imports

In [ ]:
!git clone https://github.com/tpq-classes/ai_in_finance.git
import sys
sys.path.append('ai_in_finance')


In [ ]:
!pip install git+https://github.com/yhilpisch/tpqoa

In [ ]:
import math
import tpqoa
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
%matplotlib inline

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
np.random.seed(100)
import tensorflow as tf
tf.random.set_seed(100)

In [ ]:
# ajdust path
# oanda = tpqoa.tpqoa('../../../data/oanda.cfg')

## The Data

In [ ]:
instrument = 'EUR_USD'
start = '2018-01-01'
end = '2019-03-31'
granularity='M30'
price = 'A'
fn = f'/content/ai_in_finance/oanda_{instrument}_{start}_{end}_{granularity}_{price}.csv'
fn

In [ ]:
%%time
try:
    raw = pd.read_csv(fn, index_col=0, parse_dates=True)
except:
    raw = oanda.get_history(instrument, start, end, granularity, price)
    raw.to_csv(fn)

In [ ]:
raw.info()

In [ ]:
data = raw.copy()
data['r'] = np.log(data['c'] / data['c'].shift(1))
data['d'] = np.where(data['r'] > 0, 1, 0)
#data['c-o'] = data['c'] - data['o']
#data['cc'] = np.where(data['c'] - data['o'] > 0, 1, 0)
#data['h-l'] = data['h'] - data['l']
#data['h-o'] = data['h'] - data['o']
#data['o-l'] = data['o'] - data['l']
#data['h-c'] = data['h'] - data['c']
#data['c-l'] = data['c'] - data['l']
data['v1'] = data['c'].rolling(20).std()
data['v2'] = data['c'].rolling(100).std()
data['sma1'] = data['c'].rolling(20).mean()
data['sma2'] = data['c'].rolling(100).mean()
data['ewma'] = data['c'].ewm(2 / (90 + 1)).mean()
data['diff'] = data['sma1'] - data['ewma']
data.dropna(inplace=True)

In [ ]:
del data['complete']

In [ ]:
data.head()

In [ ]:
def normalize(x, mu, std):
    return (x - mu) / std

In [ ]:
features = list(data.columns)

In [ ]:
exclude = ['r', 'd']

In [ ]:
split = int(len(data) * 0.70)
val_size = int(split * 0.15)
train = data.iloc[:split]
mu = train.mean()
std = train.std()

In [ ]:
std

In [ ]:
cols = []
lags = 10
for f in features:
    for lag in range(1, lags + 1):
        col = f'{f}_lag_{lag}'
        if f not in exclude:
            data[col] = normalize(data[f].shift(lag), mu[f], std[f])
        else:
            data[col] = data[f].shift(lag)
        cols.append(col)
data.dropna(inplace=True)

In [ ]:
# cols

In [ ]:
len(data)

In [ ]:
train = data.iloc[:split]
val = train[-val_size:]
train = train[:-val_size]
test = data.iloc[split:].copy()

## Model Fitting

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint

### Simple Example

In [ ]:
a = np.arange(25)
a

In [ ]:
a = a.reshape((len(a), -1))

In [ ]:
a.shape

In [ ]:
a[:5]

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
fn = 3

In [ ]:
g = TimeseriesGenerator(a, a, length=fn, batch_size=4)

In [ ]:
#for _ in g:
#    print(_)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=12, kernel_size=2, activation='relu',
                 input_shape=(fn, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mse', 'mae'])

In [ ]:
model.summary()

In [ ]:
%%time
model.fit(g, epochs=25, steps_per_epoch=5, verbose=False);

In [ ]:
res = pd.DataFrame(model.history.history)

In [ ]:
res.tail(3)

In [ ]:
ax = res.plot(figsize=(10, 6), style=['--', '--'])

In [ ]:
model.predict(g)

In [ ]:
model.predict(g).round()

### Financial Example

In [ ]:
len(cols)

In [ ]:
np.random.seed(100)
tf.random.set_seed(100)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=96, kernel_size=4, activation='relu',
                 input_shape=(len(cols), 1)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5)) # , seed=100))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
callbacks = [ModelCheckpoint(filepath='/content/ai_in_finance/.weights.h5',
                             monitor='val_accuracy',
                             verbose=0,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='auto')]

In [ ]:
%%time
model.fit(np.atleast_3d(train[cols]), train['d'],
          epochs=15, batch_size=48, verbose=False,
          validation_data=(np.atleast_3d(val[cols]), val['d']),
          callbacks=callbacks);

In [ ]:
res = pd.DataFrame(model.history.history)

In [ ]:
res.tail(3)

In [ ]:
ax = res.plot(figsize=(10, 6), style=['--', '--', '-', '-'])
x = np.arange(len(res));
reg = np.polyfit(x, res['val_accuracy'], deg=2)
plt.plot(x, np.polyval(reg, x), 'r');

## Backtesting

In [ ]:
# model.load_weights('.weights.h5')

In [ ]:
model.evaluate(np.atleast_3d(test[cols]), test['d'])

In [ ]:
test['p'] = np.where(model(tf.convert_to_tensor(np.atleast_3d(test[cols]), dtype=tf.float32), training=False).numpy() > 0.5, 1, 0)
test['p'] = np.where(test['p'] > 0, 1, -1)

In [ ]:
test['s'] = test['p'] * test['r']

In [ ]:
# test.head()

In [ ]:
test[['r', 's']].sum().apply(np.exp)

In [ ]:
test['p'].value_counts()

In [ ]:
(test['p'].diff() != 0).sum()

In [ ]:
test[['r', 's']].cumsum().apply(np.exp).plot(figsize=(10, 6));

## Transaction Costs

In [ ]:
pd.set_option('chained_assignment', None)

In [ ]:
pips = 0.00012

In [ ]:
ptc = pips / test['c'].mean()
ptc

In [ ]:
test['s_tc'] = test['s']
test['s_tc'][test['p'].diff() != 0] -= ptc

In [ ]:
test[['r', 's', 's_tc']].cumsum().apply(np.exp).plot(figsize=(10, 6));

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">